In [4]:
import os
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from utils import check_env_variable, check_file_exists, generate_tp_name

from gptravel.core.travel_planner.openai_engine import ChatGPTravelEngine
from gptravel.core.travel_planner.prompt import PlainTravelPrompt
from gptravel.core.travel_planner.travel_engine import TravelPlanJSON
from gptravel.core.services.geocoder import GeoCoder

from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.pydantic_v1 import BaseModel

from gptravel.core.utils.regex_tool import JsonExtractor
from utils import from_json_to_text
from typing import Dict, Any
from operator import itemgetter

load_dotenv()

check_env_variable('OPENAI_API_KEY')
examples_folder = os.path.join('examples')

OPENAI_API_KEY loaded correctly


In [3]:
PARAMETERS = {
    'departure_place': 'Milan',
    'departure_date': datetime(2024, 8, 30),
    'destination_place': 'Berlin',
    'end_date': datetime(2024, 9, 6),
    'gpt_params': {
        'model': 'gpt-4o-mini',
        'max_tokens': 600,
        'temperature': 0,
    },
}

PARAMETERS['n_travel_days'] = (PARAMETERS['end_date'] - PARAMETERS['departure_date']).days
print(PARAMETERS)

{'departure_place': 'Milan', 'departure_date': datetime.datetime(2024, 8, 30, 0, 0), 'destination_place': 'Berlin', 'end_date': datetime.datetime(2024, 9, 6, 0, 0), 'gpt_params': {'model': 'gpt-4o-mini', 'max_tokens': 600, 'temperature': 0}, 'n_travel_days': 7}


# Travel Plan Generation

In [ ]:
class Activity(BaseModel):
    """Travel Plan Activity"""
    time: dat